# eICU-CRD query API tutorial

This notebook shows examples of how to use the cyclops.query API on [eicu-CRD](https://eicu-crd.mit.edu/).

* First, setup the eICU database according to the instructions in [eicu-code](https://github.com/MIT-LCP/eicu-code/tree/master/build-db/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `EICUQuerier` accordingly.

## Imports and instantiate `EICUQuerier`

In [1]:
import cyclops.query.ops as qo
from cyclops.query import EICUQuerier

querier = EICUQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="eicu",
    user="postgres",
    password="pwd",
)
# List all tables.
querier.list_tables()

2023-03-21 11:15:50,751 INFO cyclops.query.orm - Database setup, ready to run queries!


['eicu_crd.admissiondrug',
 'eicu_crd.admissiondx',
 'eicu_crd.allergy',
 'eicu_crd.apacheapsvar',
 'eicu_crd.apachepatientresult',
 'eicu_crd.apachepredvar',
 'eicu_crd.careplancareprovider',
 'eicu_crd.careplaneol',
 'eicu_crd.careplangeneral',
 'eicu_crd.careplangoal',
 'eicu_crd.careplaninfectiousdisease',
 'eicu_crd.customlab',
 'eicu_crd.diagnosis',
 'eicu_crd.hospital',
 'eicu_crd.infusiondrug',
 'eicu_crd.intakeoutput',
 'eicu_crd.lab',
 'eicu_crd.medication',
 'eicu_crd.microlab',
 'eicu_crd.note',
 'eicu_crd.nurseassessment',
 'eicu_crd.nursecare',
 'eicu_crd.nursecharting',
 'eicu_crd.pasthistory',
 'eicu_crd.patient',
 'eicu_crd.physicalexam',
 'eicu_crd.respiratorycare',
 'eicu_crd.respiratorycharting',
 'eicu_crd.treatment',
 'eicu_crd.vitalaperiodic',
 'eicu_crd.vitalperiodic',
 'information_schema.sql_packages',
 'information_schema.sql_features',
 'information_schema.sql_implementation_info',
 'information_schema.sql_parts',
 'information_schema.sql_languages',
 'infor

## Example 1. Get all female patients discharged in 2014.

In [2]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = querier.eicu_crd.patient(ops=ops).run()
print(f"{len(patients)} rows extracted!")

2023-03-21 11:15:52,228 INFO cyclops.query.orm - Query returned successfully!
2023-03-21 11:15:52,229 INFO cyclops.utils.profile - Finished executing function run_query in 0.704464 s


43717 rows extracted!


## Example 2. Get all patient encounters with diagnoses (`schizophrenia` in `diagnosisstring`), discharged in the year 2015.

In [3]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
    ]
)
patients = querier.eicu_crd.patient(ops=ops)
diagnosis_ops = qo.Sequential(
    [
        qo.ConditionSubstring("diagnosisstring", "schizophrenia"),
    ]
)
diagnoses = querier.eicu_crd.diagnosis(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=diagnosis_ops,
).run()
print(f"{len(diagnoses)} rows extracted!")

2023-03-21 11:15:53,069 INFO cyclops.query.orm - Query returned successfully!
2023-03-21 11:15:53,070 INFO cyclops.utils.profile - Finished executing function run_query in 0.787358 s


1348 rows extracted!


## Example 3. Get potassium lab tests for patients discharged in the year 2014, for all teaching hospitals.

In [4]:
hospitals = querier.eicu_crd.hospital()
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2015),
        qo.ConditionEquals("teachingstatus", True),
    ]
)
patients = querier.eicu_crd.patient(
    join=qo.JoinArgs(join_table=hospitals.query, on="hospitalid"), ops=ops
)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("labname", "potassium"),
    ]
)
labs = querier.eicu_crd.lab(
    join=qo.JoinArgs(
        join_table=patients.query,
        on="patientunitstayid",
    ),
    ops=lab_ops,
).run()
print(f"{len(labs)} rows extracted!")

2023-03-21 11:16:03,196 INFO cyclops.query.orm - Query returned successfully!
2023-03-21 11:16:03,197 INFO cyclops.utils.profile - Finished executing function run_query in 9.992888 s


252638 rows extracted!


## Example 4. Get glucose medications (substring search) for female patients discharged in 2014.

In [5]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("hospitaldischargeyear", 2014),
        qo.ConditionEquals("gender", "Female"),
    ]
)
patients = querier.eicu_crd.patient(ops=ops)
medications_ops = qo.Sequential(
    [
        qo.ConditionSubstring("drugname", "glucose"),
    ]
)
medications = querier.eicu_crd.medication(
    join=qo.JoinArgs(join_table=patients.query, on="patientunitstayid"),
    ops=medications_ops,
).run()
print(f"{len(medications)} rows extracted!")

2023-03-21 11:16:05,470 INFO cyclops.query.orm - Query returned successfully!
2023-03-21 11:16:05,471 INFO cyclops.utils.profile - Finished executing function run_query in 2.221213 s


9063 rows extracted!
